In [1223]:
import pandas as pd
import numpy as np
df=pd.read_csv(r'bq-results-20210207-173409-w3x9p4x9871d.csv')

In [1224]:
df.columns

Index(['Title'], dtype='object')

In [1225]:
df1=pd.DataFrame()
df1['Title']=df['Title'].apply(lambda x:x.lower())
df1['Year']=df1['Title'].str.extract( "(\d{4}\D?\d*)",expand=False ).str.strip()

In [1226]:
df1["Unmatched"] = df1['Title'].apply(lambda x: x.lower())

# Specialty

In [1227]:
Keywords= ["jerseys","jersey","freshman","rookies","rc", "mvp","mvps","hof","auto","rookie","nnof","xrc","big four","bip","foil","jsy","nib","pc","opg"]

def Special(x):
    String=""
    for i in Keywords:
        for k in x.split(" "):
            if i == k:
                String += str(i) + ","
    return String[:-1]

In [1228]:
df1['Specialty']=df1['Title'].apply(lambda x: Special(x))

In [1229]:
df1.head(10)

,Title,Year,Unmatched,Specialty
0,2010-11 limited /249 keith gallon #186 rookie ...,2010-11,2010-11 limited /249 keith gallon #186 rookie ...,"auto,rookie"
1,2018-19 panini court kings acetate rookies lon...,2018-19,2018-19 panini court kings acetate rookies lon...,"rookies,rookie"
2,1974-75 topps austin carr #60,1974-75,1974-75 topps austin carr #60,
3,2012-13 preferred champs material booklet /199...,2012-13,2012-13 preferred champs material booklet /199...,hof
4,2019-20 panini absolute memorabilia blue /99 k...,2019-20,2019-20 panini absolute memorabilia blue /99 k...,rookie
5,2019 contenders variation cracked ice ticket /...,2019,2019 contenders variation cracked ice ticket /...,"auto,rookie"
6,2019 donruss optic rated rookies fanatics gree...,2019,2019 donruss optic rated rookies fanatics gree...,"rookies,rookie"
7,2019 donruss optic rated rookies fanatics gree...,2019,2019 donruss optic rated rookies fanatics gree...,"rookies,rookie"
8,2014-15 panini select concourse silver prizm j...,2014-15,2014-15 panini select concourse silver prizm j...,rookie
9,2016-17 panini studio jaylen brown #88 rookie,2016-17,2016-17 panini studio jaylen brown #88 rookie,rookie


In [1230]:
pd.DataFrame(df1.groupby('Specialty')['Specialty'].count()).rename(columns={"Specialty": "Count"}).sort_values(by='Count',ascending=False)#

,Count
Specialty,
,535132
"rc,rookie",391973
rookie,208057
auto,110089
hof,88575
...,...
"rookies,auto,auto",2
"rc,auto,rookie,foil",2
"jerseys,freshman",2


# Classifier

In [1231]:
df1['Classifier']=df1['Title'].str.extract(r'(single|lot|set|box|pack|team set|uncut sheet|box&case breaks|cello|booklet)',expand=True)
#df1['Classifier']=df1['Classifier'].str.capitalize()

In [1232]:
pd.DataFrame(df1.groupby('Classifier')['Classifier'].count()).rename(columns={"Classifier": "Count"}).sort_values(by='Count',ascending=False)#


,Count
Classifier,
lot,98601
box,46210
set,13630
pack,3536
booklet,2337
cello,1260
single,355
team set,298
uncut sheet,49


# Card Number

In [1233]:
df1['CardNumber']=df1['Title'].str.extract("#(\w*\.?\-?\/?\d?\w*\-?\w+)",expand=True )
#df1['CardNumber'].isnull().values.sum()/len(df) # 22% missing

# Quantity Made

In [1234]:
df1['QuantityMade']=df1['Title'].str.extract( "(\d?\d?\d?\d?\d?\d?\/\d+)",expand=True )
#df1['QuantityMade'].isnull().values.sum()/len(df) # 91% missing

# Professional Grader

In [1235]:
df1['Grader']=df1['Title'].str.extract(r'(psa|bgs|sgc|bvg|bccg|scd|gai|sgv)',expand=True)
#df1['Grader']=df1['Grader'].str.upper()

In [1236]:
df1['Grader'].isnull().values.sum()/len(df)  #72%missing 

0.7216182847418146

In [1237]:
pd.DataFrame(df1.groupby('Grader')['Grader'].count()).rename(columns={"Grader": "Count"}).sort_values(by='Count',ascending=False)

,Count
Grader,
psa,330981
bgs,123953
sgc,15935
bccg,1644
gai,676
bvg,423
scd,111


# TeamName

In [1238]:
#Defunct Team
import requests,re
from bs4 import BeautifulSoup
Teamname=[]
response = requests.get("https://en.wikipedia.org/wiki/List_of_defunct_National_Basketball_Association_teams")
if response.status_code == 200:
    print("Success")
else:
    print("Failure")
results_page = BeautifulSoup(response.content,'html.parser')
page = results_page.find_all("table")[1]
page = page.find_all("tr")
for i in page:
    try:
        Teamname.append(i.find('td').find('a').text)
    except:
        pass


Success


In [1239]:
#Current Teams
response = requests.get("http://www.espn.com/nba/players")
if response.status_code == 200:
    print("Success")
else:
    print("Failure")
results_page = BeautifulSoup(response.content,'html.parser')
page = results_page.find('div',{'id':'my-players-table'})
page = page.find_all("li")
for i in page:
    try:
        Teamname.append(i.find_all('a')[1].text)
    except:
        pass


Success


In [1240]:
# TeamName=Teamname+[element.upper() for element in Teamname]+[element.lower() for element in Teamname]
# pattern = '|'.join(TeamName)
# pattern

In [1241]:
TeamName = "anderson packers|packers|baltimore bullets|bullets|buffalo|chicago stags|stags|cleveland rebels|rebels|denver nuggets|nuggets|detroit falcons|falcons|indianapolis|indianapolis jets|jets|indianapolis olympians|olympians|pittsburgh ironmen|ironmen|providence steamrollers|steamrollers|sheboygan red skins|red skins|st. louis bombers|bombers|toronto huskies|huskies|washington capitols|capitols|waterloo hawks|hawks|boston celtics|celtics|brooklyn nets|nets|new york knicks|knicks|philadelphia 76ers|76ers|toronto raptors|raptors|golden state warriors|warriors|la clippers|clippers|los angeles lakers|lakers|phoenix suns|suns|sacramento kings|kings|chicago bulls|bulls|cleveland cavaliers|cavaliers|detroit pistons|pistons|indiana pacers|pacers|milwaukee bucks|bucks|dallas mavericks|mavericks|houston rockets|rockets|memphis grizzlies|grizzlies|new orleans pelicans|pelicans|san antonio spurs|spurs|atlanta hawks|hawks|charlotte hornets|hornets|miami heat|heat|orlando magic|magic|washington wizards|wizards|denver nuggets|nuggets|minnesota timberwolves|timberwolves|oklahoma city thunder|thunder|portland trail blazers|blazers|utah jazz|jazz"
TeamName = TeamName.split('|')

In [1242]:
def team_name_extract(x):
    for i in TeamName:
        if i in x.lower():
            return i

In [1243]:
df1['TeamName']=df1['Title'].apply(lambda x:team_name_extract(x))
df1['TeamName'].isnull().values.sum()/len(df) #missing 95% team name is not a frequently used key words in basketball cards

0.797173184744929

# Players

In [1244]:
import requests
from bs4 import BeautifulSoup
def fetch_famous_players(baseurl):
    famous_player = []
    response = requests.get(baseurl)
    if response.status_code == 200:
        print("Success")
    else:
        print("Failure")
    results_page = BeautifulSoup(response.content,'html.parser')
    page = results_page.find("table").find_all('td', attrs={'style':'font-family: Arial, sans-serif; color: windowtext; font-size: 10.0pt; font-weight: 400; font-style: normal; text-decoration: none; text-align: general; vertical-align: bottom; white-space: nowrap; border: medium none; padding-left: 1px; padding-right: 1px; padding-top: 1px'})
    for i in page:
        famous_player.append(i.text.split('\n')[1].strip())
    return famous_player

In [1245]:
#1-1000
famous_player = fetch_famous_players("http://ainsworthsports.com/basketball_player_rankings_all_time_1_to_1000.htm")
#1001-2000
famous_player.extend(fetch_famous_players("http://ainsworthsports.com/basketball_player_rankings_all_time_1001_to_2000.htm"))
famous_player.extend(fetch_famous_players("http://ainsworthsports.com/basketball_player_rankings_all_time_2001_to_3000.htm"))
famous_player.extend(fetch_famous_players("http://ainsworthsports.com/basketball_player_rankings_all_time_3001_to_4000.htm"))
famous_player.extend(fetch_famous_players("http://ainsworthsports.com/basketball_player_rankings_all_time_4001_to_5000.htm"))

Success
Success
Success
Success
Success


In [1246]:
famous_player.append("zion williamson")
famous_player.append("shaq o'neal")
famous_player.append("lebron")
famous_player.append("bryant")

In [1247]:
def person(x):
    for i in famous_player:
        dummy = i.lower()
        if dummy in x:
            return i

In [1248]:
df1['Player_Name']=df1['Title'].apply(lambda x: person(x))

In [1249]:
pd.DataFrame(df1.groupby('Player_Name')['Player_Name'].count()).rename(columns={"Player_Name": "Count"}).sort_values(by='Count',ascending=False)

,Count
Player_Name,
LeBron James,96458
Michael Jordan,96231
zion williamson,86535
Luka Doncic,81711
Kobe Bryant,77092
...,...
Carl Nicks,1
Quino Colom,1
Charles Edge,1


In [1250]:
import nltk
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

st = StanfordNERTagger('english.all.3class.distsim.crf.ser.gz','stanford-ner-4.2.0.jar',encoding='utf-8')
def person2(x):
    if isinstance(x['Player_Name'],str):
        return x['Player_Name']
    else:
        text = x['Title']
        for sent in nltk.sent_tokenize(text):
            tokens = nltk.tokenize.word_tokenize(sent)
            tags = st.tag(tokens)
            res = ""
            for tag in tags:
                if tag[1]=='PERSON':
                    res = res + " "+ tag[0]
        return res.strip()

In [1251]:
df1['Player_Name'].isnull().sum()/len(df1)

0.1256329688553173

In [1252]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1701703 entries, 0 to 1701702
Data columns (total 10 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   Title         object
 1   Year          object
 2   Unmatched     object
 3   Specialty     object
 4   Classifier    object
 5   CardNumber    object
 6   QuantityMade  object
 7   Grader        object
 8   TeamName      object
 9   Player_Name   object
dtypes: object(10)
memory usage: 129.8+ MB


# Manufacturer

In [1253]:
manufacturer=pd.read_csv(r'Sports_Cards_Keywords_Manufacturer.csv')
# manufacturer dictinoary
manufacturer = manufacturer.set_index('How It Appears').to_dict()['Manufacturuer']
print(len(manufacturer))
manufacturer_dict = {}
for key,val in manufacturer.items():
    if "|" in key:
        keys = key.split('|')
        for i in keys:
            manufacturer_dict[i.lower()] = val
    else:
        manufacturer_dict[key.lower()] = val
print(len(manufacturer))

443
443


In [1254]:
print(len(manufacturer_dict))

574


In [1255]:
manufacturer_list = list(manufacturer_dict.keys())
manufacturer_list = set(manufacturer_list)

In [1256]:
def manufacturer_detect(x):
    res = ""
    for i in x.split(" "):
        if i in manufacturer_list:
            res = res + i + ","
    if res:
        return res
    else:
        return np.NaN

In [1257]:
df1['Manufacturer']=df1['Unmatched'].apply(lambda x: manufacturer_detect(x))

In [1258]:
df1['Manufacturer'].isnull().sum()/len(df1)

0.1172877993398378

# Rating

In [1259]:
rating=pd.read_csv(r'Sports_Cards_Keywords_Grade.csv')
# manufacturer dictinoary
rating = rating.set_index('How It May Appear').to_dict()['Grade']
print(len(rating))
rating_dict = {}
for key,val in rating.items():
    if "|" in key:
        keys = key.split('|')
        for i in keys:
            rating_dict[i.lower()] = val
    else:
        rating_dict[key.lower()] = val
print(len(rating))

75
75


In [1260]:
print(len(rating_dict))
rating_list = list(rating_dict.keys())

1581


In [1261]:
rating_list.sort(reverse=True)

In [1262]:
def rating_detect(x):
    for i in rating_list:
        if i in x:
            return i

In [1263]:
df1['Rating']=df1['Unmatched'].apply(lambda x: rating_detect(x))

In [1264]:
df1['Rating'].isnull().sum()

1117825

In [1265]:
df1['Grader'].isnull().sum()

1227980

# Matching

In [1266]:
def manufacturer_unmatched(x):
    if isinstance(x['Manufacturer'],str):
        dummy = x['Unmatched']
        manufacturer = x['Manufacturer'].split(',')
        while("" in manufacturer): 
            manufacturer.remove("")
        for i in manufacturer:
            dummy = dummy.replace(i,"")
        return dummy 
    else:
        return x['Unmatched']

In [1267]:
df1['Unmatched']=df1.apply(lambda x: manufacturer_unmatched(x),axis = 1)

In [1268]:
def rating_unmatched(x):
    if isinstance(x['Rating'],str):
        dummy = x['Unmatched']
        dummy = dummy.replace(x['Rating'],"")
        return dummy
    else:
        return x['Unmatched']

In [1269]:
df1['Unmatched']=df1.apply(lambda x: rating_unmatched(x),axis = 1)

In [1270]:
def else_unmatched(x):
    y = x['Unmatched']
    if isinstance(x['Player_Name'],str):
        name = x['Player_Name'].lower()
        y = y.replace(name,"")
    if isinstance(x['Year'],str):
        year = x['Year'].lower()
        y = y.replace(year,"")
    if isinstance(x['Specialty'],str):
        specialty = x['Specialty'].lower().split(',')
        for i in specialty:
            y = y.replace(i,"")
    if isinstance(x['Classifier'],str):
        classifiers = x['Classifier'].lower()
        y = y.replace(classifiers,"")
    if isinstance(x['QuantityMade'],str):
        quantity_made = x['QuantityMade'].lower()
        y = y.replace(quantity_made,"")
        if "/" in y:
            y = y.replace("/","")
    if isinstance(x['Grader'],str):
        garder = x['Grader'].lower()
        y = y.replace(garder,"")
    if isinstance(x['CardNumber'],str):
        card_number = x['CardNumber'].lower()
        y = y.replace(card_number,"")
        if "#" in y:
            y = y.replace("#","")
    if isinstance(x['TeamName'],str):
        team_name = x['TeamName'].lower()
        y = y.replace(team_name,"")
    return y

In [1271]:
df1['Unmatched']=df1.apply(lambda x: else_unmatched(x),axis = 1)

In [1272]:
def unmatched_trim(x):
    strings = x['Unmatched'].strip().split(' ')
    res = ' '.join([x for x in strings if x])
    return res

In [1273]:
df1['Unmatched']=df1.apply(lambda x: unmatched_trim(x),axis = 1)

In [1274]:
# !pip install nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from collections import OrderedDict
nltk.download('punkt')
words_list = df1['Unmatched'].values.tolist()
striptext = " ".join(words_list)
words = word_tokenize(striptext)
lowercase_words = [word.lower() for word in words]
word_frequencies = FreqDist(lowercase_words)
most_frequent_words = FreqDist(lowercase_words).most_common(10000)
print(most_frequent_words)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chensiyuan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('prizm', 596766), ('mint', 219483), ('gem', 208985), ('$', 176350), ('card', 149967), ('silver', 145380), ('optic', 141342), ('prizms', 138531), ('basketball', 100870), ('upper', 96140), ('deck', 95860), (')', 88670), ('(', 88647), ('of', 77593), ('blue', 75493), ('gold', 74118), ('rated', 72593), ('red', 70587), ('treasures', 70378), ('patch', 69892), ('mosaic', 69273), ('national', 68412), ('graph', 68353), ('nba', 65232), ('hoops', 62895), ('9.5', 57415), ('signatures', 50219), ('green', 49937), ('refractor', 49625), ('jr', 44827), ('chrome', 43100), ('draft', 41523), ('contenders', 40541), ('read', 39241), ('comc', 38717), ('immaculate', 33942), ('ice', 33390), ('the', 31849), ('game', 31717), ('holo', 31260), ('purple', 30569), ('pink', 30188), ('white', 28813), ('picks', 28509), ('-', 27662), ('2', 27174), ('orange', 26022), ('break', 25669), ('premium', 25441), ('fast', 25212), ('9', 25039), ('*', 24897), ('10', 24364), ("'s", 24008), ('flawless', 22735), ('chronicles', 22730)

In [1275]:
import nltk
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

st = StanfordNERTagger('english.all.3class.distsim.crf.ser.gz','stanford-ner-4.2.0.jar',encoding='utf-8')
def person2(text):
    for sent in nltk.sent_tokenize(text):
        tokens = nltk.tokenize.word_tokenize(sent)
        tags = st.tag(tokens)
        res = ""
        for tag in tags:
            if tag[1]=='PERSON':
                res = res + " "+ tag[0]
    return res.strip()

In [1276]:
undetect_name_text= [i[0] for i in most_frequent_words]
undetect_name_text = " ".join(undetect_name_text)
undetected_name = person2(undetect_name_text).split(' ')
undetected_name.extend(["ken","griffey","allen","ginter","ohtani","shohei",
                          "alvarez","franco","lewis","jim",
                          "al","nick","kyle","yordan","gavin","mike","dominguez",
                          "rodriguez","jasson","john","ryan","andrew","alex","diego","mckenzie",
                           "carlson","howard","zac","bryan","clarke","adams","brian","pearson",
                          "alexander","andy","perez","chuck","mackenzie","hernandez","andres",'anderson',
                          "matthew","kopech","rosario","stewart","marte","lee","brandon",'dalquist',
                           "triston","tim",'dylan','larry','martin','derek','pete','abrams','foli',
                     'jamieson','jimenez','amethyst','charley','alston','christin', 'kershaw','luke',
                        'sullivan', 'dustin','shane','bush','ryne', 'mccarthy', 'alec', 'willi','hunter',
                        'samuel','dawson','kelley','jimmie','yogi','tj','rutledge','leo','johnstone',
                        'eli','giles','varsho','ronaldo','mcconnell','sonny','evan','devin','medina',
                        'yoshitomo','kindall','herrnstein','dempsey','clayton','connell','levi',
                        'derrell','jahmai','marvin','blaylock','joba','lincoln','seth','delvin','scioscia',
                        'kenny','kenney','donald','ismael','locke','opitz','janowicz','dereck','burbrink',
                        'blackmon','leron','cade','bieber','lehner','jonathon','kuhn','dahl','jonatan',
                        'jurado','dirk','steiner','kaminsky','dana','dubiel','franklyn','poppel','garko',
                        'adon','jaeckel','quinton','mccracken','ackley','peter','senzel','mason','ferrara',
                       'wander','lux','matt','wallner','spangler','julio','wright','friday','verdugo'])


In [1277]:
undetected_name = set(undetected_name)
def person_left(x):
    res = ""
    for i in x['Unmatched'].split(" "):
        if i in undetected_name:
            res = res +" "+ i
    if x['Player_Name']:
        res = x['Player_Name']+","+res.strip()
        return res
    elif (not x['Player_Name']) and res:
        return res.strip()
    else:
        return np.NaN

In [1278]:
df1['Player_Name'] = df1.apply(lambda x:person_left(x),axis = 1)

In [1279]:
df1['Player_Name'].isnull().sum()/len(df1)

0.1172801599339015

In [1280]:
def num_words_unmatched(x):
    y = x.split(' ')
    while("" in y): 
        y.remove("")
    if y:
        return len(y)
    else:
        return 0

In [1281]:
def cal_matched_percentage(x):
    if x['num_words_unmatched'] == 0:
        return 1
    else:
        return 1-(x['num_words_unmatched']/x['num_words_title'])

In [1282]:
df1['Unmatched']=df1.apply(lambda x: else_unmatched(x),axis = 1)

In [1283]:
df1['Unmatched']=df1.apply(lambda x: unmatched_trim(x),axis = 1)

In [1284]:
most_frequent_words = FreqDist(lowercase_words).most_common(100)
junk_words = [i[0] for i in most_frequent_words]
junk_words = set(junk_words)
def match_junks(x):
    res = ""
    for i in x.split(' '):
        if i in junk_words:
            res = res+" "+i
    if res:
        return res.strip()
    else:
        return np.NaN

In [1285]:
df1['Junks']=df1['Unmatched'].apply(lambda x: match_junks(x))

In [1286]:
def replace_junks(x):
    res = x['Unmatched']
    if isinstance(x['Junks'],str):
        for i in x['Junks'].split(' '):
            if i in res:
                res = res.replace(i,"")
    return res

In [1287]:
df1['Unmatched']=df1.apply(lambda x: replace_junks(x),axis = 1)

# Calculating Stats

In [1288]:
df1['Unmatched']=df1.apply(lambda x: unmatched_trim(x),axis = 1)

In [1289]:
df1['num_words_unmatched']=df1['Unmatched'].apply(lambda x: num_words_unmatched(x))

In [1290]:
df1['num_words_title']=df1['Title'].apply(lambda x: len(x.split(' ')))

In [1291]:
df1['matched_percentage']=df1.apply(lambda x: cal_matched_percentage(x),axis = 1)

In [1292]:
df1['matched_percentage'].mean()

0.8479747399303285

In [1293]:
# df1['Specialty']=df1['Specialty'].str.upper()
# df1['Classifier']=df1['Classifier'].str.capitalize()
# df1['Grader']=df1['Grader'].str.upper()

In [1294]:
def manufacturer_back(x):
    if isinstance(x,str):
        arr = x.split(',')
        res = ""
        while("" in arr) : 
            arr.remove("")
        while(" " in arr) : 
            arr.remove(" ")
        for i in arr:
            res = res + " "+ manufacturer_dict[i.strip()]
        return res
    else:
        return np.NaN

In [1295]:
def rating_back(x):
    if isinstance(x,str):
        return rating_dict[x]
    else:
        return np.NaN

In [1296]:
def lower_back(x):
    if isinstance(x,str):
        return x.lower()
    else:
        return np.NaN

In [1297]:
df1['Manufacturer']=df1['Manufacturer'].apply(lambda x: manufacturer_back(x))
df1['Rating'] = df1['Rating'].apply(lambda x: rating_back(x)) 

In [1298]:
df1.to_csv('basketball_df.csv', index=False)